# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

# Your solution to Q3 here

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

# Test code below this

In [6]:
type(w)

pandas.core.frame.DataFrame

In [7]:
w.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

In [8]:
len(b.race)

2435

In [9]:
len(w.race)

2435

In [10]:
#data.info()

print("Number of black resumes that received call",sum(b.call))

print("Number of white resumes that received call",sum(w.call))

Number of black resumes that received call 157.0
Number of white resumes that received call 235.0


# Using permutation and mean test statistic

ub = proportion of resumes with black sounding names that got a call

uw = proportion of resumes with white sounding names that got a call

H0: ub = uw

Ha: ub < uw

alpha = 0.05

In [11]:
#define function that applies the proportion test statistic to sampled data


def test_statistic(black_calls, white_calls, function):
    black_statistic = function(black_calls)
    white_statistic = function(white_calls)
    diff_statistic = white_statistic - black_statistic
    return diff_statistic

In [12]:
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

In [13]:
#define function that gives required number of permutation replicates

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates
    
    

In [14]:
def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

In [45]:
#get replicates and compute statistic for hypothesis test

# Compute difference of mean impact force from experiment: empirical_diff_means
empirical_diff_means = diff_of_means(w.call, b.call)

# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(w.call, b.call,
                                 diff_of_means, size=1000000)

# Compute p-value: p
p = np.sum(perm_replicates >= empirical_diff_means) / len(perm_replicates)

# Print the result
print('p-value =', p)

p-value = 2.2e-05


95% confidence level

In [28]:
np.percentile(perm_replicates, [2.5, 97.5])

array([-0.01560576,  0.01478439])

In [23]:
empirical_diff_means

0.03203285485506058

In [44]:
estimate_proportion_white = np.mean(w.call)
print(estimate_proportion_white)

estimate_proportion_black = np.mean(b.call)
print(estimate_proportion_black)

0.09650924056768417
0.0644763857126236


### frequentist approach

In [37]:
standard_error_white = np.sqrt( (estimate_proportion_white) * (1 - estimate_proportion_white)/(len(w.call)))

standard_error_black = np.sqrt( (estimate_proportion_black) * (1 - estimate_proportion_black)/(len(b.call)))

standard_error_diff_of_mean = np.sqrt( standard_error_white**2 + standard_error_black**2)

z_statistic = empirical_diff_means /standard_error_diff_of_mean
print(z_statistic)

p_value = stats.norm.sf(z_statistic)
p_value

4.115550519002299


1.931281906454845e-05

95% confidence interval

In [43]:
z_statistic_95CI = stats.norm.ppf(0.975)

confidence_interval = [-1*z_statistic_95CI*standard_error_diff_of_mean, +1*z_statistic_95CI*standard_error_diff_of_mean]
confidence_interval

[-0.015255126027012737, 0.015255126027012737]

# Discuss with Ryan, how this test could be done with bootstrapping. 

We need to being by assuming null hypothesis is true. For population mean, usually this is accomplished by shifting the means. However, I would like to use population proportion. It does not make sense to shift calls in each race by the proportion calls in the pooled dataset